# regression model : ver 1.0

 - X(1) -> Y
 - 손실함수(loss function) : 오차 반환 함수
 - 모델 최적화 알고리즘 : 경사하강법 알고리즘(GD, Adam) 
   - 모델 학습 : 최적의 기울기, 절편 -> loss값이 0에 수렴

In [1]:
# version 1 사용
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:

# 1. X, Y data 생성
# x_data = np.array([1,2,3])
# y_data = np.array([2,4,6])

# x변수 값이 큰 경우 : inf
x_data = np.array([1,2,3,125]) # data 정규화
y_data = np.array([2,4,6,256]) # data 정규화

# 0 ~ 1
x_data = x_data/x_data.max() # [0.0078125, 0.015625 , 0.0234375, 1.       ]
# np.log
y_data = np.log(y_data) # [-0.36651292,  0.32663426,  0.58319808,  1.71292862]


# 2. X, Y 변수 정의
X = tf.placeholder(dtype=tf.float32, shape=[None]) # x_data 공급
Y = tf.placeholder(dtype=tf.float32, shape=[None]) # y_data 공급

# 3. a, b 변수 정의
a = tf.Variable(tf.random_normal([1])) # 기울기
b = tf.Variable(tf.random_normal([1])) # 절편

# 4. 식 정의
model = tf.multiply(X,a) + b # y_pred : 회귀방정식

err = Y - model # 오차

loss = tf.reduce_mean(tf.square(err)) # 손실함수

# 5. 최적화 객체
optimizer = tf.train.GradientDescentOptimizer(0.1) # 학습율 : 0.1
train = optimizer.minimize(loss) # 손실함수 최소화 : 최적의 기울기와 절편 추출

init = tf.global_variables_initializer()

In [14]:
# 6. 반복학습
with tf.Session() as sess:
    sess.run(init) # 변수 초기화
    a_val, b_val = sess.run([a,b])
    print("최초 기울기와 절편")
    print("a = {}, b = {}".format(a_val,b_val))
    
    feed_data = {X:x_data, Y:y_data}
    
    # 반복학습 50회
    for step in range(50):
        _, loss_val = sess.run([train, loss], feed_dict = feed_data)
        if (step+1)%10 == 0:
            print("step = ", (step+1), "\n loss = ",loss_val)
        
    # model test
    y_pred = sess.run(model, feed_dict={X:[0.024]}) # 2.5
    print("y_pred = ",y_pred)
    


최초 기울기와 절편
a = [0.1223039], b = [0.63285536]
step =  10 
 loss =  1.520682
step =  20 
 loss =  0.8135233
step =  30 
 loss =  0.47816792
step =  40 
 loss =  0.3088615
step =  50 
 loss =  0.22330247
y_pred =  [1.5158052]


 - y변수 : 1칼럼
 - x변수 : 2~4칼럼
 - model 최적화 알고리즘 : GD -> Adam
 - model 평가 : MSE

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [16]:
# 1. 공급 data 생성
iris = pd.read_csv("C:/IITT/6_Tensorflow/data/iris.csv")
cols = list(iris.columns)
x_data = iris[cols[1:4]]
y_data = iris[cols[0]]
x_data.shape # (150,3)

x_train,x_test,y_train,y_test = train_test_split(x_data,y_data,test_size=0.3)


# 2. X,Y 변수 정의
X = tf.placeholder(dtype=tf.float32, shape=[None,3])
Y = tf.placeholder(dtype=tf.float32, shape=[None])


# 3. a(w), b 변수 정의 : 난수 초기값
a = tf.Variable(tf.random_normal(shape=[3,1]))
b = tf.Variable(tf.random_normal(shape=[1]))


# 4. model 생성
model = tf.matmul(X,a) + b
err = Y - model
loss = tf.reduce_mean(tf.square(err))
opt = tf.train.AdamOptimizer(0.5)
train = opt.minimize(loss) # loss 최소화 객체


In [17]:

# 5. model 학습 -> model 최적화(최적의 a,b update됨)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) # a,b 초기화
    feed_data = {X:x_train, Y:y_train}
    
    # 반복학습 100회
    for step in range(100):
        _, loss_val = sess.run([train,loss], feed_dict=feed_data)
       # print("step = {}, loss = {:.5f}".format(step+1,loss_val))
    
    # model 최적화 : 최적화된 기울기와 절편 설정
    a_up, b_up = sess.run([a,b])
    print("수정된 기울기 : {}, 절편 : {} ".format(a_up, b_up))
    
    # 테스트용 공급 data
    feed_data_test = {X:x_test, Y:y_test}
        
    # Y(정답) vs model(예측치) -> model의 a,b가 최적화되어 있는 상태
    y_true = sess.run(Y, feed_dict={Y:y_test}) # 정답
    y_pred = sess.run(model, feed_dict={X:x_test}) # 예측치
    mse = mean_squared_error(y_true, y_pred)
    
    print("MSE = ",mse)
    

수정된 기울기 : [[ 0.7078158 ]
 [ 0.26032847]
 [-0.38143966]], 절편 : [3.144752] 
MSE =  0.51447624
